In [1]:
import requests
import json
from csv import writer
import pandas as pd

#GLOBAL VARIABLES
uniswapV2 = "0x7a250d5630b4cf539739df2c5dacb4c659f2488d"
uniswapV3 = "0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45"

# change file here
addressLists = pd.read_csv("tokens.csv")
walletsAndVersions = {}

for i, row in addressLists.iterrows():
    token = row["TokenPair"]

    variables = {"input": token}
    query = """
            query($input: String!){
                swaps(
                    where: {
                        pair_: {id: $input}
                    }
                    orderDirection: asc
                    orderBy: timestamp
                    first: 1000
                ) {
                    sender
                    to
                }
            }
        """

    resp = requests.post('https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2',
                           json={'query': query, 'variables': variables})

    json_data = json.loads(resp.text)
    
    json_data = json_data["data"]["swaps"]

    for eachData in json_data:
        if(eachData["sender"] == uniswapV2 and eachData["to"] != uniswapV2):
            walletsAndVersions[eachData["to"]] = 2
        
        elif (eachData["sender"] == uniswapV3 and eachData["to"] != uniswapV3):
            walletsAndVersions[eachData["to"]] = 3

with open('wallets.csv', 'a') as fd:
    for key in walletsAndVersions:
        fd.write(key + "," + str(walletsAndVersions[key]) + "\n")

In [ ]:
wallets = pd.read_csv("wallets.csv")
wallets = wallets["Wallet"]
wallets = wallets.drop_duplicates()

weiDivisible = 1000000000000000000

for user in wallets:
    url = f"https://api.etherscan.io/api?module=account&action=balance&address={user}&tag=latest&apikey=7YRRHUFSDSM5BXCIM8FY3NPJ192WYKRC1G"
    response = requests.get(url)
    jsonData = json.loads(response.text)
    data = float(jsonData["result"])
    data = data / weiDivisible
    

    with open('balance.csv', 'a') as fd:
        fd.write(f"{user}, {data}\n")